In [1]:
# What we want to do: Retrieve a message from Flask, requesting a planet belonging to a user, and then doing some lightkurve manipulations to it, sending it back to the frontend (and allowing nfts to be minted with properties then). Potentially Flask can be moved onto Deepnote, with inporting/exporting features to/from Github

In [2]:


df_1 = _deepnote_execute_sql("""
""", 'SQL_1A0CBA44_CA08_4A3B_BFC9_E0FF2214DAB3')
df_1

In [3]:
# API Creation
!pip install flask
!pip install requests

# Connect to backend
!pip install psycopg2

# Graphing/Data components
!pip install matplotlib
!pip install lightkurve

# Connecting to the blockchain
!pip install thirdweb-sdk

  Using cached Jinja2-3.1.2-py3-none-any.whl (133 kB)
  Using cached MarkupSafe-2.1.2-cp39-cp39-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (25 kB)
  Attempting uninstall: MarkupSafe
    Found existing installation: MarkupSafe 2.0.1
    Not uninstalling markupsafe at /shared-libs/python3.9/py-core/lib/python3.9/site-packages, outside environment /root/venv
    Can't uninstall 'MarkupSafe'. No files were found to uninstall.
  Attempting uninstall: Jinja2
    Found existing installation: Jinja2 2.11.3
    Not uninstalling jinja2 at /shared-libs/python3.9/py-core/lib/python3.9/site-packages, outside environment /root/venv
    Can't uninstall 'Jinja2'. No files were found to uninstall.
You should consider upgrading via the '/root/venv/bin/python -m pip install --upgrade pip' command.
You should consider upgrading via the '/root/venv/bin/python -m pip install --upgrade pip' command.
  Using cached psycopg2-2.9.5.tar.gz (384 kB)
  Preparing metadata (setup.py) ... error
  error: subproces

In [4]:
from flask import Flask, jsonify, request
import requests
import psycopg2
import lightkurve as lk
from lightkurve import search_lightcurve
import matplotlib.pyplot as plt
from thirdweb import ThirdwebSDK

In [5]:
# Flask Initialisation
app = Flask(__name__)

# Lightcurve initialisation
planetTic = ""

In [6]:
# Contract Initialisation
sdk = ThirdwebSDK("mumbai")
contract = sdk.get_contract("0xed6e837Fda815FBf78E8E7266482c5Be80bC4bF9")

_receiver = ""
_tokenId = 0
_quantity = 0

In [7]:
# Flask/api routes
@app.route('/planet')
def planet():
    lc = search_lightcurve('KIC 757076', author="Kepler", quarter=3).download()
    lc = lc.remove_nans().remove_outliers()
    lc.to_fits(path='demo_lightcurve1.fits', overwrite=True) # create a function to convert this to an image
    return "Hello" # return a base variable from this lightcurve, use it to set generator function

app.run(host='0.0.0.0', port=8080)

 * Tip: There are .env or .flaskenv files present. Do "pip install python-dotenv" to use them.
 * Serving Flask app '__main__'
 * Debug mode: off
 * Running on all addresses (0.0.0.0)
 * Running on http://127.0.0.1:8080
 * Running on http://172.3.84.122:8080
Press CTRL+C to quit
172.3.150.40 - - [30/Jan/2023 13:07:23] "GET / HTTP/1.1" 404 -
172.3.150.40 - - [30/Jan/2023 13:07:24] "GET /favicon.ico HTTP/1.1" 404 -
172.3.150.40 - - [30/Jan/2023 13:07:33] "GET /planet HTTP/1.1" 200 -


In [ ]:


df_3 = _deepnote_execute_sql("""/* Save .fits to user's storage container, e.g.
import os
from supabase import create_client
from datetime import datetime, timedelta
from gotrue.exceptions import APIError

url = os.getenv("SUPABASE_URL") # from your `.env` file
key = os.getenv("SUPABASE_KEY")
supabase = create_client(url, key)

# Creating an account
email: str = ''
password: str = ''
# user = supabase.auth.sign_up(email = email, password = password)

# Authentication
session = None
try:
    session = supabase.auth.sign_in(email = email, password = password)
    supabase.postgrest.auth(session.access_token) # authenticate the client (which has RLS)
except APIError:
    print("Authentication failed")

resp = supabase.storage().from_('images').get_public_url('cebdc7a2-d8af-45b3-b37f-80f328ff54d6/69893963-c755-4a46-802b-b729c5482cd5')
print(resp)

# Upload image/file
#resp = supabase.storage().from_('images').upload('auth.py', 'auth.py', {'content-type': 'image/jpeg'})
res = supabase.storage().from_('avatars').list()
data = supabase.table('profiles').select('*').execute()
print("")
print(res)
print("")
print(session.user.id)
print('')
res = supabase.storage().from_('images').download('cebdc7a2-d8af-45b3-b37f-80f328ff54d6/69893963-c755-4a46-802b-b729c5482cd5')

supabase.auth.sign_out()
*/
""", 'SQL_1A0CBA44_CA08_4A3B_BFC9_E0FF2214DAB3')
df_3

In [8]:
# Selecting planets
@app.post('/select_planet')
def select_planet():
    data = request.get_json()
    planetId = data['planetId']
    planetName = data['planetName']
    planetTic = data['planetTic']
    sector_data = lk.search_lightcurve(planetTic, author = 'SPOC', sector = 23)
    #lc = sector_data.download()
    #lc.plot()
    return sector_data

# Show planet data on frontend
@app.post('/show_planet') # Can we do some calculation for nft revealing using this (i.e. mint nft after classification)
def show_tic():
    lc = sector_data.plot()
    return lc

@app.post('/mint-planet')
def mint_planet():
    data = request.get_json()
    _receiver = data['profileAddress']
    _tokenId = data['tokenId']
    _quantity = 1
    data = contract.call("claim", _receiver, _tokenId, _quantity)

In [9]:
data = contract.call("claim", 0xCdc5929e1158F7f0B320e3B942528E6998D8b25c, 2, 1)

ValidationError: 
Could not identify the intended function with name `claim`, positional argument(s) of type `(<class 'int'>, <class 'int'>, <class 'int'>)` and keyword argument(s) of type `{}`.
Found 1 function(s) with the name `claim`: ['claim(address,uint256,uint256)']
Function invocation failed due to no matching argument types.

Figure out what type of planet the object is (or what type of object, more accurately). Then choose a planet from  this contract to mint using the Thirdweb SDK based on object type. [![](https://badges.thirdweb.com/contract?address=0xed6e837Fda815FBf78E8E7266482c5Be80bC4bF9&theme=dark&chainId=80001)](https://thirdweb.com/mumbai/0xed6e837Fda815FBf78E8E7266482c5Be80bC4bF9?utm_source=contract_badge)

In [ ]:


df_2 = _deepnote_execute_sql("""
""", 'SQL_1A0CBA44_CA08_4A3B_BFC9_E0FF2214DAB3')
df_2

<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=ab6b31e5-13c3-4949-af38-1197d00bd4d1' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>